In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from transformers import ViTFeatureExtractor, VisionEncoderDecoderModel, AutoProcessor, AutoTokenizer

from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from torchvision import transforms
import os
import pandas as pd
from pptx import Presentation
import pdf2image
import win32com.client
import win32com.client.connect
#from pptxtopdf import convert
import torch
#poppler library path
os.environ["PATH"] = r"C:\Users\filip\OneDrive\Área de Trabalho\Cast Group\Projects\for_all_see_it_cast\poppler-24.07.0\Library\bin"

C:\Users\filip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  

In [4]:
device

device(type='cuda')

In [5]:
import os
import win32com.client
import pandas as pd

def extrair_slides_e_comentarios(diretorio):
    # Inicializa o PowerPoint
    ppt_app = win32com.client.Dispatch("PowerPoint.Application")
    ppt_app.Visible = True
        
    # Lista para armazenar os dados
    dados = []

    # Itera sobre todos os arquivos no diretório
    for arquivo in os.listdir(diretorio):
        if arquivo.endswith('.pptx'):
            pptx_path = os.path.join(diretorio, arquivo)
            apresentacao = ppt_app.Presentations.Open(pptx_path)

            # Processa cada slide
            for slide_index, ppt_slide in enumerate(apresentacao.Slides):
                # Define o caminho da imagem
                imagem_path = os.path.join(diretorio, f"{arquivo[:-5]}_slide_{slide_index + 1}.png")
                
                # Salva o slide como imagem
                ppt_slide.Export(imagem_path, 'PNG')
                
                # Extrai comentários
                comentarios = [comment.Text for comment in ppt_slide.Comments]

                # Adiciona os dados ao DataFrame
                for comentario in comentarios:
                    dados.append({
                        'nome_arquivo': arquivo,
                        'image': imagem_path.replace('\\','/'),
                        'text': comentario.replace('\n\n',' ').replace('\n',' ').strip()
                    })

            # Fecha a apresentação
            apresentacao.Close()

    # Cria um DataFrame a partir dos dados
    df = pd.DataFrame(dados)

    # Encerra o PowerPoint
    ppt_app.Quit()
    
    return df

# Exemplo de uso
diretorio_pptx = "C:\\Users\\filip\\OneDrive\\Área de Trabalho\\Cast Group\\Projects\\for_all_see_it_cast\\data\\pptx\\"
df_resultado = extrair_slides_e_comentarios(diretorio_pptx)

In [6]:
df_resultado.head()

,nome_arquivo,image,text
0,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,#PraTodosVerem Fundo: A imagem tem um fundo pr...
1,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,#PraTodosVerem Fundo: O fundo da imagem é bran...
2,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,#PraTodosVerem Fundo: O fundo da imagem é bran...
3,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,#PraTodosVerem Fundo: O fundo da imagem é divi...
4,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,#PraTodosVerem Fundo: O fundo da imagem é divi...


In [7]:
loc = "ydshieh/vit-gpt2-coco-en"

feature_extractor = ViTFeatureExtractor.from_pretrained(loc)
model = VisionEncoderDecoderModel.from_pretrained(loc)
processor = AutoProcessor.from_pretrained(loc)

model = model.to(device)

C:\Users\filip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "architectures": [
    "ViTModel"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": true,
  "transformers_version": "4.46.0"
}

Config of the decoder: <class 'transformers.mod

In [8]:
#dataset = load_dataset("ybelkada/football-dataset", split="train")

In [9]:
#dataset
df_resultado.head()

,nome_arquivo,image,text
0,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,#PraTodosVerem Fundo: A imagem tem um fundo pr...
1,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,#PraTodosVerem Fundo: O fundo da imagem é bran...
2,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,#PraTodosVerem Fundo: O fundo da imagem é bran...
3,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,#PraTodosVerem Fundo: O fundo da imagem é divi...
4,Decima_Jornada_da_Agilidade_Cast.pptx,C:/Users/filip/OneDrive/Área de Trabalho/Cast ...,#PraTodosVerem Fundo: O fundo da imagem é divi...


In [10]:
def image_read(image_path):
        return Image.open(image_path).convert("RGB")

In [11]:
df_resultado['image_rgb'] = df_resultado.image.apply(image_read)

In [12]:

class ImageTextDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 1]
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)

        text = self.data.iloc[idx, 1]

        return image, text

In [13]:
dataset = ImageTextDataset(df_resultado[['image_rgb','text']], transform=transforms.ToTensor())
# dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [14]:
class ImageCaptioningDataset(Dataset):
    def __init__(self, dataframe, processor, feature_extractor):
        self.dataframe = dataframe
        self.processor = processor
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item = self.dataframe.iloc[idx]
        img_path = item['image']
        text = item['text']

        image = Image.open(img_path)
        encoding = self.feature_extractor(images=image, return_tensors="pt")
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        encoding["text"] = text

        return encoding

# Example usage with your DataFrame 'df_resultado'
train_dataset = ImageCaptioningDataset(df_resultado, processor, feature_extractor)


In [15]:
# Lora configuration
config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query","value"],
)

model = get_peft_model(model, config)

## Trainable Params
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params, all_param = 0, 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad: trainable_params += param.numel()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable %: {100 * trainable_params / all_param}")

In [16]:
print_trainable_parameters(model)

trainable params: 589824 || all params: 239785728 || trainable %: 0.2459796105963404


In [17]:
def collator(batch):
    processed_batch = {}
    for key in batch[0]:
        if key != "text":
            processed_batch[key] = torch.stack([example[key] for example in batch])
        else:
            text_inputs = processor([example["text"] for example in batch], padding=True, return_tensors="pt")
            processed_batch["input_ids"] = text_inputs["input_ids"]
            processed_batch["attention_mask"] = text_inputs["attention_mask"]
    return processed_batch

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=3, collate_fn=collator)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)
epochs = 100

In [18]:
loss_list = []
model.train()
for epoch in range(1, epochs+1):
    for idx, batch in enumerate(train_dataloader):
        input_ids = batch.pop("input_ids").to(device)
        pixel_values = batch.pop("pixel_values").to(device, torch.float16)

        outputs = model(pixel_values=pixel_values, labels=input_ids)
        loss = outputs.loss
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    if epoch % 10 == 0: print(f"Epoch {epoch} done!, Loss: {loss.item()}")
    loss_list.append(loss.item()) 

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


Epoch 10 done!, Loss: 5.834036827087402
Epoch 20 done!, Loss: 5.328901767730713
Epoch 30 done!, Loss: 5.839601516723633
Epoch 40 done!, Loss: 5.753589153289795
Epoch 50 done!, Loss: 5.513179302215576
Epoch 60 done!, Loss: 5.513029098510742
Epoch 70 done!, Loss: 5.155308246612549
Epoch 80 done!, Loss: 5.544610500335693
Epoch 90 done!, Loss: 5.214959621429443
Epoch 100 done!, Loss: 5.739976406097412


In [23]:
tokenizer = AutoTokenizer.from_pretrained(loc)

def predict(image):

    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values.to('cuda')

    with torch.no_grad():
        output_ids = model.generate(pixel_values, max_length=16, num_beams=4, return_dict_in_generate=True,
  pad_token_id=tokenizer.eos_token_id).sequences

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]

    return preds

In [24]:
# with Image.open('C:\\Users\\filip\\OneDrive\\Imagens\\vinijr.png') as image:
#     preds = predict(image)
image = Image.open("C:\\Users\\filip\\OneDrive\\Área de Trabalho\\Cast Group\\Projects\\for_all_see_it_cast\\data\\pptx\\Jornada da Agilidade Cast - com PTV 1_slide_2.png").convert("RGB")

In [25]:
preds = predict(image)

In [26]:
preds

['a black and white photo of a black and white photo of a black and']